In [10]:
# Import Libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats
from scipy.signal import lombscargle
from sklearn import svm

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Loading Data
data = pd.read_csv('userData.csv')
data.head()

,user,age,gender,section,duration,arousal,valence,samples,rr
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 790, 795, 769, 768, 754, 762, 766, 774, ..."
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 749, 724, 692, 701, 724, 757, 769, 777, ..."
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 751, 735, 741, 749, 760, 752, 761, 774, ..."
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 765, 778, 757, 768, 774, 775, 773, 777, ..."
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 741, 791, 814, 813, 827, 849, 870, 820, ..."


In [3]:
# Identify the start and end indices for each user's activities
data['rr'] = data['rr'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Define the start and end of each user's data
start_indices = data.index[data['section'] == 'Rest_Start'].tolist()
end_indices = data.index[data['section'] == 'Rest_End'].tolist()

# Assuming that there's always a 'Rest_End' for each 'Rest_Start'
# Segment the data for each user
users_data = [data.iloc[start:end+1] for start, end in zip(start_indices, end_indices)]

# Now let's take a look at the first segment to verify
users_data[0].head()

,user,age,gender,section,duration,arousal,valence,samples,rr
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 790, 795, 769, 768, 754, 762, 766, 774, ..."
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 749, 724, 692, 701, 724, 757, 769, 777, ..."
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 751, 735, 741, 749, 760, 752, 761, 774, ..."
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 765, 778, 757, 768, 774, 775, 773, 777, ..."
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 741, 791, 814, 813, 827, 849, 870, 820, ..."


In [11]:
# Changing window size from the paper as the activites dont match and it is better we adjust our wondows to our activites 
# Determine the window size and overlap based on the shortest activity duration

def to_cumulative_milliseconds(rr_intervals):
    # Convert to cumulative milliseconds
    return np.cumsum(rr_intervals).tolist()

SHORTEST_ACTIVITY = data['duration'].min()  # The shortest duration in seconds
FREQUENCY_OF_INTEREST = 0.04  # The low-frequency interest rate in Hz
PERIOD_OF_LOWEST_FREQUENCY = 1 / FREQUENCY_OF_INTEREST  # The period corresponding to the frequency

# Update the rr_intervals in the dataset to be cumulative milliseconds
for user_data in users_data:
    user_data['rr'] = user_data['rr'].apply(to_cumulative_milliseconds)

# Now, let's define a windowing function that creates windows correctly
def create_windows(cumulative_rr, window_length_ms, overlap_ms):
    windows = []
    start = 0
    while start < cumulative_rr[-1]:
        end = start + window_length_ms
        window = [rr for rr in cumulative_rr if start <= rr < end]
        if window:  # Only add the window if it's not empty
            windows.append(window)
        start += window_length_ms - overlap_ms
    return windows

# Constants for the windowing process
WINDOW_LENGTH_MS = 60 * 1000  # Example: 60 seconds in milliseconds
OVERLAP_MS = 5 * 1000  # Example: 5 seconds in milliseconds




In [12]:
# Apply the function to create windows for each segment of activity for each user

for i, user_data in enumerate(users_data):
    user_data['rr_windows'] = user_data['rr'].apply(lambda rr: create_windows(rr, WINDOW_LENGTH_MS, OVERLAP_MS))
    
    # # For debugging: let's see if the windows are being created correctly
    # if not user_data['rr_windows'].empty:
    #     print(f"User {i} windows:")
    #     print(user_data['rr_windows'].iloc[0])  # Print the first set of windows for the user
    # else:
    #     print(f"User {i} has no RR windows.")

In [6]:
users_data[0].head()

,user,age,gender,section,duration,arousal,valence,samples,rr,rr_windows
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 1573, 2368, 3137, 3905, 4659, 5421, 6187...","[[783, 1573, 2368, 3137, 3905, 4659, 5421, 618..."
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 1488, 2212, 2904, 3605, 4329, 5086, 5855...","[[739, 1488, 2212, 2904, 3605, 4329, 5086, 585..."
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 1501, 2236, 2977, 3726, 4486, 5238, 5999...","[[750, 1501, 2236, 2977, 3726, 4486, 5238, 599..."
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 1518, 2296, 3053, 3821, 4595, 5370, 6143...","[[753, 1518, 2296, 3053, 3821, 4595, 5370, 614..."
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 1439, 2230, 3044, 3857, 4684, 5533, 6403...","[[698, 1439, 2230, 3044, 3857, 4684, 5533, 640..."


In [13]:
# Function to calculate HR from RR intervals
def calculate_hr(rr_intervals):
    # Assuming rr_intervals are the times of R-peaks in milliseconds
    rr_diffs = np.diff(rr_intervals)  # Calculate differences between consecutive R-peaks
    # HR for each interval is 60 seconds * 1000 ms divided by the interval in ms to get bpm
    hr = 60 * 1000 / rr_diffs
    return hr.tolist()  # Convert to list for easier handling

# Apply the function to calculate HR for each window of RR intervals
for i, user_data in enumerate(users_data):
    user_data['hr_windows'] = user_data['rr_windows'].apply(lambda windows: [calculate_hr(window) for window in windows])



In [14]:
users_data[0].head()

,user,age,gender,section,duration,arousal,valence,samples,rr,rr_windows,hr_windows
0,20230608_183012_3_02001743,42,0,Rest_Start,185,3,3,215,"[783, 2356, 4724, 7861, 11766, 16425, 21846, 2...","[[783, 2356, 4724, 7861, 11766, 16425, 21846, ...","[[38.14367450731087, 25.33783783783784, 19.126..."
1,20230608_183012_3_02001743,42,0,Reading,185,4,2,207,"[739, 2227, 4439, 7343, 10948, 15277, 20363, 2...","[[739, 2227, 4439, 7343, 10948, 15277, 20363, ...","[[40.32258064516129, 27.124773960217, 20.66115..."
2,20230608_183012_3_02001743,42,0,Breath_1,130,5,3,155,"[750, 2251, 4487, 7464, 11190, 15676, 20914, 2...","[[750, 2251, 4487, 7464, 11190, 15676, 20914, ...","[[39.973351099267155, 26.833631484794275, 20.1..."
3,20230608_183012_3_02001743,42,0,Stroop,190,3,0,220,"[753, 2271, 4567, 7620, 11441, 16036, 21406, 2...","[[753, 2271, 4567, 7620, 11441, 16036, 21406, ...","[[39.52569169960474, 26.13240418118467, 19.652..."
4,20230608_183012_3_02001743,42,0,ReactionTime,227,5,3,241,"[698, 2137, 4367, 7411, 11268, 15952, 21485, 2...","[[698, 2137, 4367, 7411, 11268, 15952, 21485, ...","[[41.69562195969423, 26.905829596412556, 19.71..."


In [9]:
# Constants for LF and HF bands in Hz
LF_BAND = (0.04, 0.15)
HF_BAND = (0.15, 0.4)

# Function to perform spectral analysis and extract LF and HF power
def extract_frequency_powers(rr_intervals, sampling_rate=4):
    # Convert rr_intervals to time in seconds
    time = np.cumsum(rr_intervals) / 1000.0  # Convert milliseconds to seconds
    # Normalize RR intervals by subtracting the mean
    rr_normalized = rr_intervals - np.mean(rr_intervals)
    # Define frequency range for analysis
    freqs = np.linspace(0.01, 1.0, 10000)
    # Compute the Lomb-Scargle periodogram (power spectral density estimate)
    pgram = lombscargle(time[:-1], rr_normalized[:-1], freqs)
    # Calculate the power in the LF and HF bands
    lf_power = np.trapz(pgram[(freqs >= LF_BAND[0]) & (freqs < LF_BAND[1])], freqs[(freqs >= LF_BAND[0]) & (freqs < LF_BAND[1])])
    hf_power = np.trapz(pgram[(freqs >= HF_BAND[0]) & (freqs < HF_BAND[1])], freqs[(freqs >= HF_BAND[0]) & (freqs < HF_BAND[1])])
    return lf_power, hf_power

# Apply the function to each window of RR intervals
for user_data in users_data:
    user_data[['lf_power', 'hf_power']] = user_data['rr_windows'].apply(
        lambda windows: [extract_frequency_powers(window) for window in windows]
    ).apply(pd.Series)

ValueError: Columns must be same length as key